In [58]:
import praw
import time

import pandas as pd
import pandas_gbq

In [56]:
! conda install pandas-gbq --channel conda-forge --y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/homebrew/Caskroom/miniforge/base/envs/dev1

  added / updated specs:
    - pandas-gbq


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blosc-1.21.1               |       hd414afc_3          30 KB  conda-forge
    boost-cpp-1.78.0           |       h1cb353e_1        16.4 MB  conda-forge
    branca-0.6.0               |     pyhd8ed1ab_0          27 KB  conda-forge
    cairo-1.16.0               |    h73a0509_1014         1.3 MB  conda-forge
    cfitsio-4.2.0              |       h2f961c4_0         731 KB  conda-forge
    click-plugins-1.1.1        |             py_0           9 KB  conda-forge
    cligj-0.7.2                |     p

In [15]:
from datetime import datetime
from pytz import timezone
tz = timezone('EST')


datetime.datetime(2022, 11, 29, 7, 11, 31, 260302, tzinfo=<StaticTzInfo 'EST'>)

In [ ]:
reddit = praw.Reddit(
    client_id = os.getenv("REDDIT_CLIENT_ID"),
    client_secret = os.getenv("REDDIT_SECRET_KEY"),
    user_agent='ABot/0.0.1'
    )

In [28]:
def get_reddit_comments(nfl_team_subreddit_name: str):
    """
    Description: 
        Gets 5 top comments from 5 hottest posts (so max 25 comments)
    Args: 
        nfl_team_subreddit_name: name of nfl team subreddit
    Returns: 
        List of reddit comments
    """
    subreddit = reddit.subreddit(nfl_team_subreddit_name)
    subs = []

    # get top 5 hottest posts
    hot = subreddit.hot(limit=5)
    for submission in hot:
        subs.append(submission)
    
    res = []
    for submission in subs:
        print("-----------------")
        print("reddit.config.reddit_url" + f"{submission.permalink}")
        submission.comments.replace_more(limit=0)
        submission.comment_limit = 5
        i = 0
        for top_level_comment in submission.comments:
            if i < 5:
                comment = top_level_comment.body
                # strip all whitespace characters
                comment = " ".join(comment.split())
                res.append(comment)
            else:
                break
            i += 1
        print("-----------------")
    
    return res
        

    





In [11]:
nfl_team_subreddits = {
    "NFC": {
        "north": ["ravens","bengals", "browns", "steelers"],
        "south": ["texans","colts","jaguars","tennesseetitans"],
        "east": ["buffalobills","miamidolphins","patriots","nyjets"],
        "west": ["denverbroncos","kansascitychiefs","raiders", "chargers"]
    },
    "AFC": {
        "north": ["chibears","detroitlions","greenbaypackers","minnesotavikings"],
        "south": ["falcons","panthers","saints","buccaneers"],
        "east": ["cowboys","nygiants","eagles","commanders"],
        "west": ["azcardinals","losangelesrams","49ers","seahawks"]
    }
}

In [ ]:
final_df = pd.DataFrame({'team_name': pd.Series(dtype='str'),
                   'upload_date': pd.Series(dtype='datetime64[ns]'),
                   'comment': pd.Series(dtype='str')})

In [14]:
# iterates through all 32 teams
for conference in ["NFC", "AFC"]:
    for division in ["north","south", "east", "west"]:
        # 4 teams in a division
        for i in range(4):
            nfl_team_subreddit_name = nfl_team_subreddits[conference][division][i]
            print(nfl_team_subreddit_name)

            comments = get_reddit_comments(nfl_team_subreddit_name = nfl_team_subreddit_name)
            upload_date = [datetime.now(tz).replace(hour=5, minute=0)]*len(comments)
            team_name = [nfl_team_subreddit_name]*len(comments)

            concat_this = {"team_name": team_name,"upload_date": upload_date,"comment": comments}

            final_df = pd.concat([final_df, pd.DataFrame(concat_this)], ignore_index=True)

            print(f"Done loading comments for r/{nfl_team_subreddit_name}!...")
            # to help with api rate limit
            time.sleep(3)

ravens
bengals
browns
steelers
texans
colts
jaguars
tennesseetitans
buffalobills
miamidolphins
patriots
nyjets
denverbroncos
kansascitychiefs
raiders
chargers
chibears
detroitlions
greenbaypackers
minnesotavikings
falcons
panthers
saints
buccaneers
cowboys
nygiants
eagles
commanders
azcardinals
losangelesrams
49ers
seahawks


In [26]:
final_df = pd.DataFrame({'team_name': pd.Series(dtype='str'),
                   'upload_date': pd.Series(dtype='datetime64[ns]'),
                   'comment': pd.Series(dtype='str')})

In [71]:
from transformers import pipeline

# Allocate a pipeline for sentiment-analysis
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [75]:
final_df["sentiment"] = final_df.apply(lambda row: classifier(row.comment), axis = 1)

final_df["label"] = final_df.apply(lambda row: row.sentiment[0]["label"], axis = 1)
final_df["score"] = final_df.apply(lambda row: row.sentiment[0]["score"], axis = 1)

In [34]:
for nfl_team_subreddit_name  in ["eagles","kansascitychiefs"]:
    comments = get_reddit_comments(nfl_team_subreddit_name = nfl_team_subreddit_name)
    upload_date = [datetime.now(tz).replace(hour=5, minute=0)]*len(comments)
    team_name = [nfl_team_subreddit_name]*len(comments)

    append_this = {"team_name": team_name,"upload_date": upload_date,"comment": comments}

    final_df = pd.concat([final_df, pd.DataFrame(append_this)], ignore_index=True)

    print(f"Done loading comments for r/{nfl_team_subreddit_name}!...")
    # to help with api rate limit
    time.sleep(3)
    

-----------------
reddit.config.reddit_url/r/eagles/comments/z7qva3/tuesday_nfl_discussion_thread/
-----------------
-----------------
reddit.config.reddit_url/r/eagles/comments/z7c2q5/more_people_should_sync_merrill_and_mike_on/
-----------------
-----------------
reddit.config.reddit_url/r/eagles/comments/z7493k/hurts_this_is_my_heisman_pose_lane_help/
-----------------
-----------------
reddit.config.reddit_url/r/eagles/comments/z6xkxh/slay_goes_on_nsfw_rant_after_scott_and/
-----------------
-----------------
reddit.config.reddit_url/r/eagles/comments/z7d6vs/one_year_ago_today_the_progress_is_insane_how_is/
-----------------
Done loading comments for r/eagles!...
-----------------
reddit.config.reddit_url/r/KansasCityChiefs/comments/z7s397/daily_discussion_november_29_2022/
-----------------
-----------------
reddit.config.reddit_url/r/KansasCityChiefs/comments/z7iv6l/patrick_bronze_lavon_mahomes_iii_has_arrived/
-----------------
-----------------
reddit.config.reddit_url/r/Kansas

In [60]:
final_df.dtypes

team_name      object
upload_date    object
comment        object
dtype: object

In [86]:
for n in range(20,30):
    print(final_df.iloc[n].comment)
    print(final_df.iloc[n].sentiment)
    print("\n")

I’m glad someone finally put the foot down on Moore returning. I’d be curious who finally made the call but we’ll never hear it. Dude has showed promise as a WR lately but just cannot track the ball well on punts or body catch. Idk if it’s the yips or what. 

Maybe eventually he would learn the skill set. And maybe it would help him as a receiver overall. He has the potential I think to be a monster on returns. But it’s too late in the season to be dealing with that shit.

Not to be negative and shit on him. I realize that skill set is valuable to a receiver though. It’s just not the time anymore. Ignoring injuries on kick offs for this take.
[{'label': 'NEGATIVE', 'score': 0.9690434336662292}]


The broncos are averaging the least amount of ppg this century. Less than the expansion Texans, winless browns and winless lions. The legion of boom gave up like .2 more ppg so it’s like they are literally playing the legion of boom every week. Fuck the broncos
[{'label': 'NEGATIVE', 'score': 

In [108]:
final_df.dtypes

team_name       object
upload_date     object
comment         object
sentiment       object
label           object
score          float64
dtype: object

In [69]:
" ".join(comments[18].splitlines())

'The browns are coming.   You have to poop?      I was dead at that one.'

In [70]:
" ".join(comments[6].split())


'RemindMe! 24 years 5 months Patrick "Bronze" Mahomes III NFL draft'

In [63]:
comments[18]

'The browns are coming. \n\nYou have to poop?   \n\n\nI was dead at that one.'

In [62]:
pandas_gbq.to_gbq(final_df, "nfl.comments_example", if_exists='replace',
table_schema = [{'name':'upload_date','type': 'DATE'}])

100%|██████████| 1/1 [00:00<00:00, 5356.71it/s]


In [119]:
final_df.sample(2)

,team_name,upload_date,comment,sentiment,label,score
33,kansascitychiefs,2022-11-29 05:00:40.054450-05:00,Good call. He's impressed me at WR the last fe...,"[{'label': 'NEGATIVE', 'score': 0.978076517581...",NEGATIVE,0.978077
11,eagles,2022-11-29 05:00:37.039420-05:00,I never would've guessed that Slay talks like ...,"[{'label': 'POSITIVE', 'score': 0.999816358089...",POSITIVE,0.999816


In [138]:
final_df["team_name"] = final_df["team_name"].astype("str")
final_df["comment"] = final_df["comment"].astype("str")
final_df["upload_date"] = final_df["upload_date"].astype("datetime64[ns]")

final_df["sentiment"] = final_df["sentiment"].astype("List")
final_df["label"] = final_df["label"].astype("str")

TypeError: data type 'List' not understood

In [122]:
final_df.dtypes

team_name              object
upload_date    datetime64[ns]
comment                object
sentiment              object
label                  object
score                 float64
dtype: object

In [103]:
import pandasql

In [104]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [124]:
tmp_df = final_df.loc[:, ~final_df.columns.isin(["sentiment"])]

In [139]:
tmp_df.sample(2)

,team_name,upload_date,comment,label,score
10,eagles,2022-11-29 10:00:37.039420,Slay is fucking hilarious,POSITIVE,0.999765
8,eagles,2022-11-29 10:00:37.039420,Lane came out and said the grass saved his kne...,POSITIVE,0.898162


In [145]:
# ARRAY_AGG(DISTINCT label) AS labels
q = """
WITH b AS (
    SELECT *,
    IIF(label = "POSITIVE",1,-1) AS sign
    FROM tmp_df
)
SELECT team_name, AVG(score*sign) AS sentiment_summary
FROM b
GROUP BY team_name


"""

names = pysqldf(q)
names

,team_name,sentiment_summary
0,eagles,-0.000465
1,kansascitychiefs,-0.044626


In [96]:
final_df.sample(2)

,team_name,upload_date,comment,sentiment,label,score
19,eagles,2022-11-29 05:00:37.039420-05:00,I think this is the first time he's had the sa...,"[{'label': 'POSITIVE', 'score': 0.998314380645...",POSITIVE,0.998314
14,eagles,2022-11-29 05:00:37.039420-05:00,The man is definitely authentic.,"[{'label': 'POSITIVE', 'score': 0.999868869781...",POSITIVE,0.999869


In [101]:
final_df.groupby(by=["team_name"]).agg(a=("label","value_counts")).reset_index()
#.agg({"label":{"a":"value_counts"}}).reset_index()

,team_name,label,a
0,eagles,NEGATIVE,10
1,eagles,POSITIVE,10
2,kansascitychiefs,NEGATIVE,12
3,kansascitychiefs,POSITIVE,11


In [89]:
final_df.columns

Index(['team_name', 'upload_date', 'comment', 'sentiment', 'label', 'score'], dtype='object')

In [154]:
final_df.loc[:, ~final_df.columns.isin(["sentiment", "label", "score"])].to_csv("gs://gcf-sources-134756275535-us-central1/nfl_comments.csv", index = False)

In [155]:
pd.__version__

'1.5.0'

In [161]:
from transformers import pipeline

from pandasql import sqldf

# set up pysqldf
pysqldf = lambda q: sqldf(q, globals())

# Allocate a pipeline for sentiment-analysis
classifier = pipeline('sentiment-analysis')

# read data
input_comments_df = pd.read_csv("gs://gcf-sources-134756275535-us-central1/nfl_comments.csv")

# TODO: figure out how to retain types when transferring to and from GCP
# cast to right types to make pandasql work
input_comments_df["team_name"] = input_comments_df["team_name"].astype("str")
input_comments_df["comment"] = input_comments_df["comment"].astype("str")
input_comments_df["upload_date"] = input_comments_df["upload_date"].astype("datetime64[ns]")

# classify sentiment
input_comments_df["sentiment"] = input_comments_df.apply(lambda row: classifier(row.comment), axis = 1)
input_comments_df["label"] = input_comments_df.apply(lambda row: row.sentiment[0]["label"], axis = 1)
input_comments_df["score"] = input_comments_df.apply(lambda row: row.sentiment[0]["score"], axis = 1)

# drop unncessary column
input_comments_df.drop("sentiment", axis = 1, inplace=True)

# cast to right types to make pandasql work

input_comments_df["label"] = input_comments_df["label"].astype("str")


q = """
WITH b AS (
    SELECT *,
    IIF(label = "POSITIVE",1,-1) AS sign
    FROM input_comments_df
)
SELECT team_name, AVG(score*sign) AS sentiment_summary
FROM b
GROUP BY team_name


"""

final_stats = pysqldf(q)



input_comments_df.to_csv("gs://gcf-sources-134756275535-us-central1/nfl_comments_w_sentiment.csv", index = False)
final_stats.to_csv("gs://gcf-sources-134756275535-us-central1/nfl_comments_agg_stats.csv", index = False)




No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


AttributeError: 'function' object has no attribute '__version__'

In [156]:
input_comments_df = pd.read_csv("gs://gcf-sources-134756275535-us-central1/nfl_comments.csv")


In [165]:
pd.read_csv("gs://gcf-sources-134756275535-us-central1/nfl_comments_agg_stats.csv")


,team_name,sentiment_summary
0,eagles,-0.000464
1,kansascitychiefs,-0.044626


In [157]:
display(input_comments_df.dtypes)
display(final_df.dtypes)

team_name      object
upload_date    object
comment        object
dtype: object

team_name              object
upload_date    datetime64[ns]
comment                object
sentiment              object
label                  object
score                 float64
dtype: object

In [148]:
final_df.to_csv("gs://gcf-sources-134756275535-us-central1/nfl_comments_w_sentiment.csv")

Unnamed: 0      int64
team_name      object
upload_date    object
comment        object
dtype: object

# Testing input to front end

In [3]:
import pandas as pd
stats_df_uri = "gs://gcf-sources-134756275535-us-central1/nfl_comments_agg_stats.csv"
df = pd.read_csv(stats_df_uri)

In [10]:
df

,team_name,sentiment_summary
0,49ers,0.511841
1,azcardinals,0.036550
2,bengals,-0.159469
3,browns,-0.198943
4,buccaneers,0.008278
5,buffalobills,0.341729
6,chargers,0.072469
7,chibears,-0.277598
8,colts,-0.116950
9,commanders,-0.164114


In [ ]:
{
"49ers":"SF",
"azcardinals":"ARI",
"bengals":"CIN",
"browns":"CLE",
"buccaneers":"TB",
"buffalobills":"BUF",
"chargers":"LAC",
"chibears":"CHI",
"colts":"IND",
"commanders":"WAC",
"cowboys":"DAL",
"denverbroncos":"DEN",
"detroitlions":"DET",
"eagles":"PHI",
"falcons":"ATL",
"greenbaypackers":"GB",
"jaguars":"JAX",
"kansascitychiefs":"KC",
"losangelesrams":"LAR",
"miamidolphins":"MIA",
"minnesotavikings":"MIN",
"nygiants":"NYG",
"nyjets":"NYJ",
"panthers":"CAR",
"patriots":"NE",
"raiders":"OAK",
"ravens":"BAL",
"saints":"NO",
"seahawks":"SEA",
"steelers":"PIT",
"tennesseetitans":"TEN",
"texans":"HOU",
}

In [23]:
for x in df.sort_values(by="team_name").team_name.values:
    print(x, len(x))

49ers 5
azcardinals 11
bengals 7
browns 6
buccaneers 10
buffalobills 12
chargers 8
chibears 8
colts 5
commanders 10
cowboys 7
denverbroncos 13
detroitlions 12
eagles 6
falcons 7
greenbaypackers 15
jaguars 7
kansascitychiefs 16
losangelesrams 14
miamidolphins 13
minnesotavikings 16
nygiants 8
nyjets 6
panthers 8
patriots 8
raiders 7
ravens 6
saints 6
seahawks 8
steelers 8
tennesseetitans 15
texans 6


In [11]:
df.sort_values(by=["sentiment_summary"], ascending = False)

,team_name,sentiment_summary
0,49ers,0.511841
17,kansascitychiefs,0.499036
23,panthers,0.462026
5,buffalobills,0.341729
29,steelers,0.190590
26,ravens,0.169420
18,losangelesrams,0.164793
20,minnesotavikings,0.111701
13,eagles,0.083841
19,miamidolphins,0.082785
